In [1]:
import pandas as pd
import numpy as np

In [2]:
debug=0

⚠️ Attention à fournir un fichier XLSX

In [14]:
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/19-LNS1U4-LNS1T2-eCampus-Majeure.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/19-Session2-LNS1U4-LNS1T2-eCampus-Mineures-Final.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/19-LNS1U4-LNS1T2-eCampus-Mineures.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/19-Session2-LNS1U4-LNS1T2-eCampus-Majeure.xlsx"
# notesNom=u"/Users/gilles/Downloads/20-LNS1T2-Mineures.xlsx"
# notesNom=u"/Users/gilles/Downloads/20-LNS1T2-FAD.xlsx"
# notesNom=u"/Users/gilles/Downloads/20-LNS1U4-Majeure.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/NotesCours/21-Session1/21-LNS1U4-LNS1T2-eCampus-Mineures.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/NotesCours/21-Session1/21-LNS1U4-LNS1T2-eCampus-FAD.xlsx"
# notesNom=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/NotesCours/21-Session1/21-LNS1U4-LNS1T2-eCampus-Majeure.xlsx"
notesNom=u"/Users/gilles/sDrive/Cours/Bordeaux/L1-LinguistiqueGenerale/NotesCours/24-Session1/24-1LNSU4.xlsx"

In [15]:
xls = pd.ExcelFile(notesNom)
xls.sheet_names

[u'Notes']

In [16]:
# dfNotes=pd.read_excel(notesNom,encoding="utf8")
dfNotes = pd.read_excel(xls, sheet_name=u'Notes',encoding="utf8")
dfNotes[u"Numéro d’identification"]=dfNotes[u"Numéro d’identification"].astype("int").astype("str")
dfNotes.set_index([u"Numéro d’identification"],inplace=True)
len(dfNotes.columns)

23

In [17]:
dfNotes.columns=[u'Nom', u'Prénom', u'Adresse de courriel',u'Prasi', u'Yomig V', u'Yomig T', u'Kartin V', u'Kartin T', u'Nemas V', u'Nemas T', u'CoupeMot', u'CoupeMorphe']+sorted([u"Q%02d"%(n+1) for n in range(1,11)])+["date"]

# dfNotes["Q02"]

#### sélection des colonnes
- notesCC pour le contrôle continu
- notesQuizz pour les quizz en CM

In [18]:
notesCC=[u'Prasi', u'Yomig V', u'Yomig T', u'Kartin V', u'Kartin T', u'Nemas V', u'Nemas T',
         u'CoupeMot', u'CoupeMorphe'
        ]
# notesCC=u"Morpho1 Morpho2 Bizar1 Bizar2 Phono Sem".split(" ")
# notesQuizz=sorted([u"Quizz%d"%(n+1) for n in range(2,8)]+[u"Quizz7b"])
notesQuizz=sorted([u"Q%02d"%(n+1) for n in range(1,11)])
# dfNotes=dfNotesXLS[notesCC+notesQuizz]
dfNotes[notesQuizz].head(10)

,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10,Q11
Numéro d’identification,,,,,,,,,,
450977,-,-,-,-,11,-,13,15,12,-
448262,20,20,14,16,14,7,13,13,13,10
450285,20,20,16,19,13,7,20,15,17,10
441237,-,-,-,-,-,-,-,-,-,-
445328,20,13,11,20,13,0,13,15,13,12
451963,14,11,13,16,18,-,13,13,18,12
450988,-,-,-,-,-,-,-,-,-,-
416094,20,20,16,20,13,0,13,13,15,18
450742,17,14,16,19,0,7,12,15,13,15


### Calcul des notes avec bonification
Toutes les notes du CC (*nbCC*) comptent et on inclue aussi les *nbCC* meilleures notes des Quizz en partant de la meilleure une par une
- *notes* contient toutes les moyennes calculées
- on retourne le max de *notes*

In [19]:
def moyenneCC(x):
    if debug: print x.name
    xNotesCC=sorted([n for n in x[notesCC].tolist() if not (n=="-" or np.isnan(n))],reverse=True)
    sommeNotes=float(sum(xNotesCC))
    nbNotes=len(notesCC)
    moyenneInitiale=round(sommeNotes/nbNotes+.25)
    return moyenneInitiale
        

In [20]:
def bonus(x):
    if debug: print x.name
    xNotesCC=sorted([n for n in x[notesCC].tolist() if not (n=="-" or np.isnan(n))],reverse=True)
    sommeNotes=float(sum(xNotesCC))
    nbNotes=len(notesCC)
    moyenneInitiale=round(sommeNotes/nbNotes+.25)
    xNotesBonus=sorted([n for n in x[notesQuizz].tolist() if not (n=="-" or np.isnan(n))],reverse=True)
    if debug: print "Quizz",xNotesBonus
    if debug: print "CC",xNotesCC
    if xNotesCC:
        notes=[]
        nbCC=len(xNotesCC)
        bonusNotes=xNotesBonus
        if len(bonusNotes)>nbCC:
            bonusNotes=bonusNotes[:nbCC]
        notes.append(moyenneInitiale)
        if debug: print "somme", sommeNotes,"moyenne brut",sommeNotes/nbNotes
        if debug: print "moyenne initiale",moyenneInitiale,"bonus",bonusNotes
        for b in bonusNotes:
            sommeNotes+=b
            nbNotes+=1
            nouvelleMoyenne=round(float(sommeNotes)/nbNotes+.25)
            notes.append(nouvelleMoyenne)
            if debug: print b,sommeNotes,nbNotes, nouvelleMoyenne
        if debug: print notes
        return max(notes)
    else:
        return np.nan
        

### Ajout de la colonne **Notes** à la structure et exportation
- apply(lambda x: bonus(x), axis=1) permet d'appliquer la fonction bonus à chaque ligne
 - la fonction bonus reçoit une ligne de données complète
 - les valeurs sont accessibles par colonne
- on ajoute 3 colonnes
 - Note pour la note de contrôle continu avec les bonus de quizz
 - NoteCC pour la note des exercices en ligne brute
 - Gain pour la différence entre les deux notes

In [21]:
dfNotes.loc[:,"Note"]=dfNotes.loc[:,notesCC+notesQuizz].apply(lambda x: bonus(x),axis=1)
dfNotes.loc[:,"NoteCC"]=dfNotes.loc[:,notesCC].apply(lambda x: moyenneCC(x),axis=1)

In [22]:
dfNotes["Gain"]=dfNotes["Note"]-dfNotes["NoteCC"]

In [23]:
dfNotes["Gain"].mean()

1.8235294117647058

In [24]:
dfNotes[dfNotes["Note"].notnull()][u"Prénom Nom Note NoteCC Gain".split(" ")]

,Prénom,Nom,Note,NoteCC,Gain
Numéro d’identification,,,,,
448262,HERINIAINA,ANDRIAMANALINTSOA,2.0,0.0,2.0
450285,MERVAN,APAYDIN,18.0,17.0,1.0
445328,CLEMENT,AUGEZ,15.0,14.0,1.0
451963,JADE,AVENTURIN,6.0,1.0,5.0
416094,ELENA-ROSE,BEL,16.0,15.0,1.0
450742,ELOISE,BELVISI,8.0,5.0,3.0
450554,MAELYS,BERTEL--BOUSBAINE,15.0,13.0,2.0
451572,EMILY,BOUCARD,19.0,18.0,1.0
446920,MARYLOU,BOUFFENIE,13.0,10.0,3.0


In [25]:
with pd.ExcelWriter(notesNom,mode='a',engine='openpyxl') as writer: 
    dfNotes[u"Prénom Nom Note NoteCC Gain".split(" ")].to_excel(writer, sheet_name=u'eCampus',encoding="utf8")